In [ ]:
from fastapi import FastAPI, HTTPException, Request, Response, Depends
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
import os
import base64
import httpx
from tempfile import TemporaryDirectory
from shutil import unpack_archive
from pathlib import Path
from concrete.ml.deployment import FHEModelServer
import asyncio
import nest_asyncio
import json

nest_asyncio.apply()

app = FastAPI()
security = HTTPBearer()

CONTAINER_ID = os.getenv("CONTAINER_ID")
MODEL_ID = os.getenv("MODEL_ID")
SERVER_PORT = 80  # Default port for HTTP
AUTH_SERVER_URL = os.getenv("AUTH_SERVER_URL")
CLIENTZIP_BASE64 ="..."
SERVERZIP_BASE64 = "UEsDBBQAAAAIAGuCB1lGgkWElwIAAJMSAAAZAAAAY29tcGlsYXRpb25fZmVlZGJhY2suanNvbu1Y32+bMBB+718R8bRJacAGY9y3NGFa1KyJEqqpaqOIUK/1CjYCZ20W9X+fIeQHlbqs2YqWKjwg7PvOd/fZ58OeH9VqWsCSYMrkJ0pvJn5wn2ontSvVX6vN83eGSGSbBiKKRcokEzztfetNZTyVa2z2XI2Kz1F9qRnRSCSzi9S/pX2adEWgFOYrBS0UwYdrTb9IaZLq3+8mus/9QPAb39Qp/5Hqk5DxmyjUQzbR45m8E9xsOLpygh7HylU1aqorfJBQSXWFE5w/6iJOxyyKw0Y8u9ZOgIHRifFR2QUIQ8Mw6tWahyZcmIfEwRjAas1DAhbWsYmxXXHskFhF6CYwkelUa90EhXWkJh2ZdrXWCXSK2IFBgP089im/5+KB58vSwgg6hfhplTncj6iSapHPuLbqTaUvWSpZUM68dUpl2SqmXGYDO2RjwpXgns7KannSlloKNXRbA9fT6s/6jVJ7tNEalYyo6Pxsj8h8r26NlVwQMU1WPrS6bnMw/nLR9Tr9bqfV9Dq9c22FfqofONzOoXveGlz2Pbc9brbbndcxaJs78HfmXo6HXzte6/O+cLgsMy+TuBHT25J32ut5Q2/Q7L8f7vqnwz8nDRnWu0/Zori91bZ3oPAvdz0M3z1/xR/OtiVYEXe/qRhgT/nbpWDsxN3LBWNfqXtVvfiXuQr/T8LeuFwcGNxSLY6eRaBJIf2ww7MrlCH7SXMOjbK0uGEpxBunxGy0fCQtu5IJ6SOTs/yEbxELYGKAXHYbiokf9t0kEYmSkgbCCJtQHUexY6kDqUOPwQIZLzG4YROCbRtgAykAdojC5H+gC49OhZCpTPz4TE1w4Re0HNNGCr+GZdIHJoO7DZgNkYXIIsQFakgzujcglgGRffT0C1BLAwQUAAAACABrggdZId/b9AMAAAABAAAADAAAAGlzX3NpbXVsYXRlZDMAAFBLAwQUAAAACABrggdZT/zLJQYAAAAEAAAAFgAAAGNvbXBvc2l0aW9uX3J1bGVzLmpzb27LK83JAQBQSwMEFAAAAAgAa4IHWYgT+QgeFAAAiIkAAA8AAABzaGFyZWRsaWIuZHlsaWLtXQ94U1WWvy9pmxRakvI3QAtZDFocpCkghnGURIvcrEE7UrSrQBralKS0aUxTAT9dWst8Q77YWWbc/cZ1Z1dm9FO/WWdlRtd/49hWWkC/3Rn8N+usO0sV/zxFBXVEUCB7zr335c9r0rSI7ux+7+h795137z3vnHPPOb/7Agm/++LDMwZCJD0hpAgOMxyOQkK+x64ImQnHaxIhXm/dyvo6MpKc2fnCtAPJ6436t0ZTw9TyzGW8vXZOii9Il1tAutJZr7cj2rmxI6e8hirRpvFS2nyjkFeUnN/YHupIKaiW1y/kHTCl+CKSm0BeRzQSDG3KIW/nCt7OUPEK6UbI6wxtCYaavMFQc3sWebvF/IY0Xi0jnXBdZSanxlXn8l557TVruDSnGJCj1YsDZZvZ/E3tacuaU16XMsDJ5yukxFxqvnodMuV1CXl20d+lkpfNTsrkeNzXXL2yxq34TFHImcnvLOatRDLjpRSOABwW1f2qW3yRqub21iZ/pKPq1uaqWxYHmjZGNy5f1tq0dNmyQMfNkebo5m3LHXagTaGquqpoWzgYbqoOLO7wVnUEfBF/U2tw46KmbXBGeUtBE7PQxwhOXiJ44x2E3E2431GHA8CfD+1D3yNkGBwwEa5rhU4TxFECx3SVf9VUCfOtWe7PFs9597mrKrb7HHc95fwP499O+83jiwkfz30ATyibwK7ZGkhL9Reqnoc6OIQsHo8XFOLsqs6OSBXYjMeabR1Rf9uiK1Je4POcGfM4OSNhXzSAs65sDzVG/FF/qy+06brOUDTY5k8TUA5+axByqtZ24Oq0BDZW+UI+iK8m35Iqf+iWjqqNrZBSba1MkfC2aKA9tGSRo6ojGPVfFPY1bvZt8ndUNYrnVHHpouScL+zcDetQCe0CweO6aKSRRhpppJFGGmmkkUb/N2it6wbX9a61rro1tPuDu+HNhsb+3R17xR37nMZus5lp/DHbMbi7svp4dUK+bA4hffafFhJ5JVy5Y0M0tpf27F1Bew7309hrRyY273In9tFCI7ytunvOVGy/oPp52n0SX3q3v9NsmlsD4vcXHMDn0sSA6PnrD4684Om9+PAiQjyxz+WXK6AzvsM2zHSpsVXSWKvNindkccdCe9ntHbYGvAOMFZmAYCzIhCWU9oxtK2vvt3Wx9mHbTibjJ7ZdbPC0SbPQtATtvZPJguc/iy931f3yVRWKqTdWoKmDFKztGQJT3wZT5SNm09wutMOd2K/YmxiXvZ9exO09XI5tq81OY+tsDk+syeZEiylo57QI7Tw2e5pq/8xmHJV/DW2f46WJRH6iPNtqlJrmOjP0u3w8+q0R+l0p9KtF/epRvwbULwD6maeBPh+ierVp6pWwCe/K5Un1LEy9QXdsv7tn/wp3z7v9ntiJI5P48xNDXD/ac3pc+j26kOt332yuXxj128rWmvC130nYGn97Cqj1OmoZTtOyeTZ34i2zFS0js4UTM9bZ0sxokWnuDsL12Kvo++W49NULfT+dxSPSLCLUIvF4P8Yic4fNKnF7jBiulezmwzY7D9d7TclwNWeG649ncac/OEsx575ZOHUvWOQGc9xgjgfMKWbx4KiVMB6M21fR2ED1S0emptv3HO0e4AZ+UbT9YPVx2n2GGfi2Yt/HfNw+0SHsO+9b3L4pqEm8CfW/37aLcP3vZu0fMfN2E56JD/HVua00adIukmHS4Exu0ivQ9nXdUUrkgzOFSYPunkEw6S0wafjI9Iz1YfY5u8Zmn2Ec9l15IbdvyUxeQfYQvo5P4kNjR9HKfmHaAW7ax8Vo2odo2p5M096zoIyj8kkLmqabTOTPLChlvyr4WD3F/HBi/p5epcSbMWe8GZP63rWA69tj4fo6RCV0quKNijisFaFWz0PtT4Wo/FuovCMz1BYygYfk7ySVdzDl96lCjee3OzEI60F0oP+pUdbDmbkexlzrkbRv/qOVPDzum8H1PyhC7TXm6sdsw2IxZL4Yb+l5rbqTDYSJijnr2fyE3AZt306njsjBGWItMqqpqTmtXjF1TxvHsR7z9ULfT6dzfY8Rvi4nhL5EIJSR+38HU/tONixN20em81XtgxYA+VfQDJ2qLHxnOi5qjc08dKrMCWhYD9nnoPuduKigkpNKRtY6iylrLeI+++SSdg8i8N/kWuda79rg8q4fOrFlr7I+GfsDVlHjOgThehq/wogXtXBhxgsKFxa8cMYNEFfMgP0Ax0RcmI3iAjGZeHov+/h87vk3AEZi+2jPB0bau4NVbzR8JxsE2O6Bym2BcbGoreTpeXC3hbRItfK903ANBmFnAN3MdrrgAE30iz3DATowPIn2Tj1AewvpfHS1B7YJvQUHsN9Y/fwRq1gvmG7HxYrX2Ry0+zm+tDDESC/12CpNPT6Ju9aKEuxUAgEDdOCNiVQawDsAhjgx9jJEQs9J4+2PqPECphprTI/bjO6Bty5zD5y6rI86uToBOTGVRZqnN8CyBl3wFK6U60m2PtAT+6j6JQ+oWOuqa5EC2LcmEYWZR6cy6+1MmPzEVL5FATk0qxyKchhA0vg6W4M79gchrYXA2t4gGA9s9rA8u673xHnNdq2VffxJtbhSbrhOEwxxyQSDOBQbFmIwXuTv8GlWDPnJU7mfAywp9rJYBF+Z7jx0JpF471Y40d6KwzZWVuJRW4H8mym8xOzHtveyYRsGiPzTKSxWMTBSsbpX5e+0eO35wCGC1eqJR4ye+K1mT7zFAtFZY1uaGZqrxEUlD83519t4sl41JRWaNbD6EIqO9KB8F4JyalpQTpgirAYAhKBstZk9C96GkgH7j+VHOnymuSB1mEqvuXs95QZqch+gPW+cwLiN/ZYOvAMx9Vsae44OvAlXz1Hpk/RFjN8GT40dXx07Kdx8HXj595NFEsCuZPmRyM/Aj/9wnuLHErlzMvfj9snMj3efx/z4V3ySI92J2fyXke9OJd8dSr7blXyvhAsrc3NG4psV7xoV77LEBxVPzONqyWWjJb4FfQxPOqRK/AfKlMTHnIewMtIFAxhZkFIUYQgc6Zb2eYInJEBvKAN69tIA+QkJTU09M3WEpOUnbGjrUZSFFYfeaa+z949yHR04rO9zWnmqNsh1ZahVwwqWTFD1cPfB8gz/1JYvkeupo8/+4r89sXeqX2qRWnQB7IEL4xooQUNzCamVL+KqO/so7D9kqYxtZzxgX9jTG4YS8I5I3aKMEnCclwBKnyScnLUYAjjoBigGDfKgWRSDBkAu+edmvo8VxeCpt3TqQgD56oGXu4AHYsrOsv+wDv+0DyS26FsKhHAoDdfL3Vx0LS7X1XDN680qM9dcKK2uWaDw6th7rhtQfhhd0TIDZKI72IPENciv88TvZxtBVt1Q0RayVv7SxPK8i/Ad+ismnov/ZsooPqARPMkDcR7m/qlMlh9wikP+tUmECWj+QxMvQA00jqudGHB378Oh7u79LEZYYNzx8WlWiG60pgrRUhOP1MtNLIFusLIEspiyFiKGJ7nyp1bJH6rkj1PJH0cOvMxIG8DLgbkcL385aax4mV6a2iel0saCzmBKWFO5Uwkbs+JdiIN23JRdWofodxfbFR7E8VaGfXWsCk7gxSv2Au15E+rXQTpwUk+DL7JtdfAY86n0ewoaNNDgHwywc6abn3cAGg8x4xZAbXsBkiKMO0kodCC0nsGn4fYuVK9ehZd2BS9/V5oFL4uy4SXleFnE8bJB7itV8BIz7/ulWfCyKCde1gu8LErHyyI1Xu7ieFldmsLLmaUYskWj4WWRwMuS0hRevlmSiZdFDC8NAi9v5Hj5aEWqzveW8DD9cQkL0z0VLExvLvna8PJyFV6eX8FzdHrJ+PDyPyeOhpdRwEt3b0k57uaenu/+kQ3/p5+/jwE5IK+gL36B0dNb8CrTK/gJCzQeXMGXeBB+kr6umRBaxCH0rokZEPo34Nr68pRrF08UbzsTmWvXljPXTp84AkLF50G58p8q+e9U8t+h5L9dwc/KHIVA8baCn0/O5jo9NCF/IeD4me7z5gkjCkGyBtR6RPYK1LRy1Kw19TwCz8bPkyj7AIjthzlefmtyEi91fU6zkqq2CRl4WZ/CS102vATgCRCGDS0FiJevTkG8PF0s8DIM7zfyi8WZwLZvBLCxzEfUcUCOksBQEtUKANeUZEs8xlwkP8CF1/KSEC0eCcZZgJjhGmrbMhlhDB8wJCBNp0DaM+x1n1WJfQLSKoszIE0q5pX8pDGjPoyANHtK6ajNKcvGFKQ9a1RqRBwXhEOawLJa0x1reZH4wcxUJAeMPGoiRhbJ8Zkskj3GrEXiHNeHIvFZznHD+OrD04aMWG3FWB2C+uAJDqOxvEzQ9HqWJd89sTdTKe+qg5y/VxGb2OvuHmRug9QHPKhYb0k57BIDd5jLwBx2o4U5bLZh5O45ez09u/dlq+JES0baI/6LTyp+WTRa2leiK61Z8L9IcSVCOfNn8n0ZFqKgmO0B4M22EMtBb+HLrH6y12UzvC5PE/UNpzuwWsA7snkORpvd1LNavCNXogwHzgr2M6/C5Ibq1xm2z7l9kNUP3LUr+H454Pvlafh+tHCs+F6bge8B+VBhOr4/WDgufG8YE76L9+HvFqbwfXnhmPF9YWEK33WF2fB9jsD3e3jqvjwtFYm/KOCR+EQBi8SD7AMT+YcFXxu+V6jy96ppPH+XFYwvf7/Qfy35e0gROyJ//3Fqymtb9Nxr3XrmtXumMq/dpB9r/vZ8QNFf8b800ngEXHUFGAhGw4UVLyrhotLD9tDx6+y4ibXEZ8C5hsYtnvhyT/z8zLdgxacO7tOLPpnCweBNXbpPnehTmnr/hRHq91+dsmfC7RJYaly94CN3Yp87+CELWtMjR1ZLp1dWJ1Yv/7zDxfyJnoRdRhR0jr2/OjaMGWQVDl0L+WOXe4VQ/Pi8ewjvu6U/goBIHziVTkm9Ec3Tcacu1DGnrmQfkMhFfDoduR9yXZ8shLEvadxA49eCO0GZMhpfBa/vwKwCd4GKq8Ai5CopctthyApPfI1jdXyyJ36Tc3V8tifeQVfHlyFEwwKsw3irw7eY22xQWAtstBeMlBcehuyBaLMPnTBD6Rw6ZfbY6uGMH9LFr64dOlX2XSOc3JVDJ8pclMakZAZtwI89d9UmdQQlquHKjKrG9sufvp9I1I/S3/Ph6P33fTR6/96PR+9f86c8z/9s9P4lJ0bvv/Lk6P2PfplH/1PYTxLz74GQT8z/CTv/EzvvZuefsfP97PwAOz/Ezj9n54fZ+V/YeQ87/4qdH2Pnx9n5SXPq7yKLv7ZM8DWe8ZR8VZLydeQcMFapZyXgK1Bei8Y0eHzDvl4bx2XROer/M7JoNFXGoWb2oWdh57imZB08Vgl5xp1991gU+PPJo3MUi1JOZrwPG49C0qjsVyNpFG7M08Y1NXPs+I2RznLe2KZI3zzajIvUyql5napVj8vV5punV/FjnZerVeubyy51q9bjqz4v3/h883I9f7yk9mM+f6rpqwZtPj9IIu0MEm8M2Bj4fWwMrDGMlMsnSpKBz+NfmDOwkQb+HOXyG50n8cYw5nl8QvKOxBvDOXteDn9KvPna9Rwv5Yr/XHGc67uX+erJNz0/l5x88/Ppc66fP9bnflP65/J7ru8Wn229+qbnnSu5kigBkqgIbJ6q5YeoGOnjiBgEdGlnaHOofUso2drhv0v5uHIxMr1l378uY98XZofyfXe8xjUtIQUwjn2DXm81El1DOZHYx5hEp9MZp/Lls3xbLKO5SxKtTrR60RaItlC0RaI1iNYo2mLRThDtRNGWiLZUtJNEy79Gb+8iJBVL+F1l/j14/u3uK0bc10gjjTTSSCONNNJII4000kgjjTTSSCONNNJII4000kgjjTTSSCONNNJII4000kgjjTTSSCONNNJII4000kgjjTTSSCONNNJII4000kij/79khQN/LhB/9xF/nzP9tw+U34dQ/k1v9puo5qLUv5suxp/RwTEPjuvgAEFnHiZENwzHCeifJBFplkR0F8LRAsczcByDo1xHdK1w7NbhP9Du3dwWbvRu9EUiQX8kyTe3R7wdUV80CJfBUDDbfbgd9To607s2ext9ra3JO5ta2zf6Wr3RQMTva/KGOtugp6mZCYhEk9ftYeJtjvj9xNsGk9sbofW3RfzNXl9Tk7d1i9/bGAwH/JGof2u0w9u5bGlGf7jVFwxhl2pkxsCN7e3RjmjEF2aD0nsa28PbvO0hvzfiC21O3vWHGtub/F7/1rAvBCp0RpnZSSnJcZv92zq2BKONgRFy2zpbvY2tfl8kl2rJRfSSTXpy7EZp0aL2trA37IuQaXrSv0FaRObpye4NhBRXk4rFZMUSUrOUuC8m1ywjay4h9Q6yfjnZqCddGyBWhtfDqR9Pu/HUhafhdXgPT7vXQTx421qDEa/yj6qDp4MhUpjJ/kDSk13zuFq75u027yns1/UX7Sk8qDtYuEe3C46Duj2SVSE27r8ghCDMiH0mn3dc8MOzRfzqOW+t4PyFgm/g00mN4M3ncX6D0m/j/FbB2y/g/I+U/krOPyz4Yxdyfp/yvIWcx9+4MAEfvojzMwTvFL/bcbHgrYJfK3gi+E7BDy/i/N8Jvl/w/yr43YJ/VfBdgv9M8A2CnyaJ5wv+YsFbBb9W8PgbI0hN/AfUzijhElXxd6r4B1X8Uyr+Zc4nf0bmkIp/n/PJUnRMxZ9QjbfoMnmXir9Jxd+s4v9exT+q4vE3SHDNsP5hiM2CA0MLf4sFQ2oOHHMJr6V/AQeGL4YShs/8czDfClUqS+Zku5lK4FF7F1Xn6V+cp39Jnv6lefovztO/LE//JXn6HXn6l5MRztQg6H8Dgsa6QBlLmzsocodT7kDMHcK5gz932uROuByJSsj/AFBLAwQUAAAACABrggdZnicypDwCAABHCQAAEQAAAGNsaWVudC5zcGVjcy5qc29uxVVJj5swFP4vPjORF3CAY6eXUedQKb1Vc3DgTWIN2MiYpCnKf68NTEUWsrTq9Gaj5+dvw69Fb7CrwaK0RcUWFpAZsF/cJ5R+b5HMUYoDVAkjynoo+SxLULXUCqUkYjxAUllYgflqIJP9dx4Gvu23XQUoRUuphNmh/T7oG5LphnOc3N+PTvejOIzvb8guAGT3AHwJ/PFPWtvaGlEdyypV1dinXhHd2GFzqDdsoHjUjbJd1VLU8KxXbu1orQ6hedS62CldSlEs5E+HJCJOm40wUqjMbeczHGLOcMKjhCbzKKTwQNkUn1LnTdF0KEZLJazcuF7t3sl1ynkg9XzsqqvNdFkZqPtPqAbI0SgRv7VgYy3oLVp4Ckda0FMtCKbhWIx4FoaURowQmjCaRDGHB0Y+QAw2JUafFh+SrbTZejIteKwQmVKIjRQ6IB7OWMy5Y45jPE9wyObwQPC/J94/Fj3yv8jHAXs2xT4csWdj9mQWYc5xzBPKE8Y4/f/sLwWiEtmbVKvjULz4E9JkjbRDRvyN/dKI7ZN61R5ovRYeWIvy9+t8CQkYDphvcZ60rBdypRyA9FUUNbg66/i993QRfZTVGoyFH93UEEv/uGV2MXlZh1YrP1lguuoSJHCHd5Xt9q1XfEjC9bft3MC67up5QzocOnd++PoBqV9uZW7X3atzpF13F5x033ejoQ/DzaaRIAncPEs+zDh34S3OXcd11j16y2N8bpr/gXtKK7jPPWuay+YpUfpfvhRSuR91/wtQSwMEFAAAAAgAa4IHWZ4nMqQ8AgAARwkAACEAAABwcm9ncmFtX2luZm8uY29uY3JldGUucGFyYW1zLmpzb27FVUmPmzAU/i8+M5EXcIBjp5dR51ApvVVzcOBNYg3YyJikKcp/rw1MRRaytOr0ZqPn52/Dr0VvsKvBorRFxRYWkBmwX9wnlH5vkcxRigNUCSPKeij5LEtQtdQKpSRiPEBSWViB+Wogk/13Hga+7bddBShFS6mE2aH9PugbkumGc5zc349O96M4jO9vyC4AZPcAfAn88U9a29oaUR3LKlXV2KdeEd3YYXOoN2ygeNSNsl3VUtTwrFdu7WitDqF51LrYKV1KUSzkT4ckIk6bjTBSqMxt5zMcYs5wwqOEJvMopPBA2RSfUudN0XQoRkslrNy4Xu3eyXXKeSD1fOyqq810WRmo+0+oBsjRKBG/tWBjLegtWngKR1rQUy0IpuFYjHgWhpRGjBCaMJpEMYcHRj5ADDYlRp8WH5KttNl6Mi14rBCZUoiNFDogHs5YzLljjmM8T3DI5vBA8L8n3j8WPfK/yMcBezbFPhyxZ2P2ZBZhznHME8oTxjj9/+wvBaIS2ZtUq+NQvPgT0mSNtENG/I390ojtk3rVHmi9Fh5Yi/L363wJCRgOmG9xnrSsF3KlHID0VRQ1uDrr+L33dBF9lNUajIUf3dQQS/+4ZXYxeVmHVis/WWC66hIkcId3le32rVd8SML1t+3cwLru6nlDOhw6d374+gGpX25lbtfdq3OkXXcXnHTfd6OhD8PNppEgCdw8Sz7MOHfhLc5dx3XWPXrLY3xumv+Be0oruM89a5rL5ilR+l++FFK5H3X/C1BLAwQUAAAAAABrggdZZyESqEgAAABIAAAADQAAAHZlcnNpb25zLmpzb257ImNvbmNyZXRlLXB5dGhvbiI6ICIyLjcuMCIsICJjb25jcmV0ZS1tbCI6ICIxLjYuMSIsICJweXRob24iOiAiMy44LjEwIn1QSwECFAMUAAAACABrggdZRoJFhJcCAACTEgAAGQAAAAAAAAAAAAAApIEAAAAAY29tcGlsYXRpb25fZmVlZGJhY2suanNvblBLAQIUAxQAAAAIAGuCB1kh39v0AwAAAAEAAAAMAAAAAAAAAAAAAACkgc4CAABpc19zaW11bGF0ZWRQSwECFAMUAAAACABrggdZT/zLJQYAAAAEAAAAFgAAAAAAAAAAAAAApIH7AgAAY29tcG9zaXRpb25fcnVsZXMuanNvblBLAQIUAxQAAAAIAGuCB1mIE/kIHhQAAIiJAAAPAAAAAAAAAAAAAADtgTUDAABzaGFyZWRsaWIuZHlsaWJQSwECFAMUAAAACABrggdZnicypDwCAABHCQAAEQAAAAAAAAAAAAAApIGAFwAAY2xpZW50LnNwZWNzLmpzb25QSwECFAMUAAAACABrggdZnicypDwCAABHCQAAIQAAAAAAAAAAAAAApIHrGQAAcHJvZ3JhbV9pbmZvLmNvbmNyZXRlLnBhcmFtcy5qc29uUEsBAhQDFAAAAAAAa4IHWWchEqhIAAAASAAAAA0AAAAAAAAAAAAAAKSBZhwAAHZlcnNpb25zLmpzb25QSwUGAAAAAAcABwDLAQAA2RwAAAAA"
EVALUATIONKEY_BASE64 = "..."
async def verify_token(credentials: HTTPAuthorizationCredentials = Depends(security)):
    apikey = credentials.credentials
    headers = {"Authorization": f"Bearer {apikey}"}
    body = {"model_id": MODEL_ID, "container_id": CONTAINER_ID}
    async with httpx.AsyncClient() as client:
        response = await client.post(AUTH_SERVER_URL, headers=headers, json=body)
        if response.status_code != 200:
            raise HTTPException(status_code=401, detail="Invalid token")
        data = response.json()
        if not data.get("result"):
            raise HTTPException(status_code=401, detail="Authentication failed")
        return True


def save_base64_and_json_to_zip(client_base64, json_str, zip_path):
    with TemporaryDirectory() as tempdir:
        client_specs_path = os.path.join(tempdir, "client.specs.json")
        with open(client_specs_path, "w") as json_file:
            json.dump(json.loads(json_str), json_file)

        client_zip_path = os.path.join(tempdir, "client.zip")
        decode_and_save_base64(client_base64, client_zip_path)

        with zipfile.ZipFile(zip_path, "w") as zipf:
            zipf.write(client_specs_path, "client.specs.json")
            zipf.write(client_zip_path, "client.zip")


def save_base64_to_zip(base64_str, zip_path):
    with open(zip_path, "wb") as f:
        f.write(base64.b64decode(base64_str))


def save_base64_zip(encoded_str, output_dir, output_filename):
    # Ensure the output directory exists
    output_dir_path = Path(output_dir)
    output_dir_path.mkdir(parents=True, exist_ok=True)

    # Decode the base64 encoded string
    zip_bytes = base64.b64decode(encoded_str)

    # Define the output file path
    zip_path = output_dir_path / output_filename

    # Write the bytes to the zip file
    with open(zip_path, "wb") as zip_file:
        zip_file.write(zip_bytes)

    # Return the path to the saved zip file
    return str(zip_path)


@app.get("/test")
async def test():
    print("success")
    return Response("hello world!")


@app.get("/init")
async def get_client():
    client_zip_base64 = CLIENTZIP_BASE64
    client_zip_bytes = base64.b64decode(client_zip_base64)
    return Response(content=client_zip_bytes, media_type="application/zip")


# 이건 클라이언트에서만 실행하는것. 테스트용으로 실행함
@app.get("/make")
async def get_evaluation_key():
    # Let's create the client and load the model
    #output_dir = "/home/jovyan/output"
    output_dir = "./output"

    save_base64_zip(CLIENTZIP_BASE64, output_dir, "client.zip")
    save_base64_zip(SERVERZIP_BASE64, output_dir, "server.zip")

    fhemodel_client = FHEModelClient(output_dir, key_dir=output_dir)
    # The client first need to create the private and evaluation keys.
    serialized_evaluation_keys = fhemodel_client.get_serialized_evaluation_keys()
    with open(output_dir + "/serialized_evaluation_keys.ekl", "wb") as f:
        f.write(serialized_evaluation_keys)
    return Response(serialized_evaluation_keys)


@app.get("/predict")
async def predict():
    body = await request.json()
    evaluation_key = body.get("evaluation_key")
    encrypted_data = body.get("encrypted_data")

    server_zip_base64 = SERVERZIP_BASE64
    server_zip_bytes = base64.b64decode(server_zip_base64)
    with TemporaryDirectory() as tmpdirname:
        server_zip_path = Path(tmpdirname) / "server.zip"
        with open(server_zip_path, "wb") as f:
            f.write(server_zip_bytes)

        unpack_archive(server_zip_path, tmpdirname, "zip")
        fhe_model_server = FHEModelServer(tmpdirname)

        encrypted_prediction = fhe_model_server.run(encrypted_data, evaluation_key)

        return {"encrypted_prediction": base64.b64encode(encrypted_prediction).decode()}


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=SERVER_PORT)